In [ ]:
import pandas as pd

In [ ]:
covid = pd.read_excel('https://github.com/randall-romero/econometria/raw/master/data/Covid.xlsx', 'Covid-2020-09-15')
covid

In [ ]:
pob = pd.read_excel('https://github.com/randall-romero/econometria/raw/master/data/Covid.xlsx', 'Población-2020', header=7)

pob = pob.dropna(how='all', axis=0).dropna(how='all', axis=1)

pob

pob.drop_duplicates(subset=['Unnamed: 1'], keep='last', inplace=True, ignore_index=True)

pob = pob[~pob['Unnamed: 1'].isin(['Costa Rica', 'Guanacaste'])]

pob = pob.iloc[:, :2]

pob.columns = ['Cantón', 'Población']

#pob.set_index('Cantón', inplace=True)

pob

In [ ]:
covid.merge(pob, on='Cantón', sort=True)

In [ ]:
pob['Cantón'][1]

In [ ]:
covid['Cantón'][1]

In [ ]:
covid['Cantón'] = covid['Cantón'].str.replace(u'\xa0', u'')
covid['Provincia'] = covid['Provincia'].str.replace(u'\xa0', u'')

In [ ]:
covid = covid.query('Provincia != "Otro"')

In [ ]:
datos = pob.merge(covid, on='Cantón', sort=True, how='outer')

In [ ]:
datos.head(20)

In [ ]:
for vt, v in zip('áéíóú', 'aeiou'):
    pob['Cantón'] = pob['Cantón'].str.replace(vt, v)

In [ ]:
datos = pob.merge(covid, on='Cantón', sort=True, how='outer')

In [ ]:
datos[datos.Población.isna()]

In [ ]:
datos[datos.Acumulados.isna()]

In [ ]:
pob['Cantón'] = pob['Cantón'].str.replace('Aguirre', 'Quepos')

In [ ]:
covid['Cantón'] = covid['Cantón'].str.replace('Vazquez de Coronado', 'Vasquez de Coronado')

In [ ]:
datos = pob.merge(covid, on='Cantón', sort=True, how='inner')

In [ ]:
datos

In [ ]:
datos.set_index('Cantón', inplace=True)
datos.eval('activ_pc = 1000 * Activos / Población', inplace=True)

In [ ]:
datos

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(figsize=[12,5])
datos['activ_pc'].sort_values(ascending=False).plot.bar(ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=[6,15])
datos.sort_values('activ_pc', ascending=True, inplace=True)

ax.barh(datos.index, datos.activ_pc)

In [ ]:
datos.dropna(inplace=True)

In [ ]:
color_provincia = {provincia: f'C{i}' for i, provincia in enumerate(datos.Provincia.unique())}

In [ ]:
datos['color'] = datos.Provincia.map(color_provincia)

In [ ]:
plt.style.use('seaborn-dark')

In [ ]:
fig, ax = plt.subplots(figsize=[6,18])
datos.sort_values('activ_pc', ascending=True, inplace=True)

ax.barh(datos.index, datos.activ_pc, color=datos.color)

total = datos.sum()
ax.axvline(1000 * total.Activos / total.Población, color='white', linestyle='--', lw=2)
ax.set_ylim([-1, 81.5])

In [ ]:
total = datos.sum()
1000 * total.Activos / total.Población

In [ ]:
provinciales = datos.groupby('Provincia').sum()
provinciales.eval('activ_pc = 1000 * Activos / Población', inplace=True)
provinciales

In [ ]:
fig, ax = plt.subplots(figsize=[6,4])
provinciales.sort_values('activ_pc', ascending=True, inplace=True)

ax.barh(provinciales.index, provinciales.activ_pc)

In [ ]:
wiki = pd.read_html('https://en.wikipedia.org/wiki/Cantons_of_Costa_Rica')[1]

In [ ]:
superficie = wiki[['Canton', 'Land area (km²)[5]']].copy()
superficie.columns=['Cantón','Superficie']

In [ ]:
for vt, v in zip('áéíóú', 'aeiou'):
    superficie['Cantón'] = superficie['Cantón'].str.replace(vt, v)

In [ ]:
datos = datos.merge(superficie, on='Cantón')

In [ ]:
datos.eval('densidad = Población / Superficie', inplace=True)
datos

In [ ]:
datos.densidad.hist()

In [ ]:
datos.plot.scatter(x='densidad', y='Activos', logx=True, logy=True)

In [ ]:
import seaborn as sns
import numpy as np

In [ ]:
datos2= datos[['densidad', 'Activos']]
sns.regplot(x='densidad', y='Activos', data=np.log10(datos2))

In [ ]:
from statsmodels.formula.api import ols

In [ ]:
ols('Activos ~ densidad', datos2).fit().summary()